In [1]:
import random
import copy

# INISIALISASI POPULASI

In [12]:
#FUNGSI INI AKAN MENGEMBALIKKAN ARRAY OF ARRAY DARI POPULASI SEBANYAK nIndividu DAN PANJANG GEN SEBANYAK nGen
def populasi(nInd,nGen):
    pop = [[random.randint(0,1) for i in range (nGen)]for i in range (nInd)]
    return pop

In [13]:
#FUNGSI INI MENGEMBALIKAN ARRAY BILANGAN REAL DARI BINER SUATU INDIVIDU
def Encoding(a): #ENCODING DENGAN BATASAN x1[-3,3] dan x2[-2,2] (BASED ON KASUS PADA SOAL)
    penyebut=0
    ipenyebut=-1
    for i in range(int(len(a)/2)):
        penyebut += 2**ipenyebut
        ipenyebut+=-1

    gn1 = 0
    gn2 = 0
    ign = -1
    ix2 = int(len(a)/2)
    
    for i in range(int(len(a)/2)):
        gn1+=a[i]*(2**ign)
        gn2+=a[ix2]*(2**ign)
        ign+=-1
        ix2+=1
    x1 = -3+(((3-(-3))/penyebut)*gn1)
    x2 = -2+(((2-(-2))/penyebut)*gn2)
    arr = [x1,x2]
    return arr

#FUNGSI INI MENGEMBALIKAN ARRAY BILANGAN REAL DARI BINER SUATU POPULASI
def PopEncoding(nInd,populasi):
    pop = []
    for i in range(nInd):
        pop.append(Encoding(populasi[i]))
    return pop

# FITNES FUNCTION

In [14]:
#FUNGSI INI AKAN MENGEMBALIKAN NILAI DARI FUNGSI
def fungsi(x1,x2):
    fung = (4-(2.1*(x1**2))+((x1**4)/3))*(x1**2)+(x1*x2)+((-4+(4*(x2**2)))*(x2**2))
    return fung
    
#FUNGSI INI AKAN MENGEMBALIKAN NILAI FITNES DARI SUATU INDIVIDU
def fitnes(x1,x2):
    fit= (4-(2.1*(x1**2))+((x1**4)/3))*(x1**2)+(x1*x2)+((-4+(4*(x2**2)))*(x2**2))
    a = 1
    fitness = 1/(fit+a)
    return fitness

#FUNGSI INI AKAN MENGEMBALIKAN NILAI FITNES DARI SUATU POPULASI
def hitungfitnes(pop):
    fitness=[]
    for i in range (nInd):
        for j in range(2):
            if j==0:
                x1=pop[i][j]
            elif j==1:
                x2=pop[i][j]
        if(x1<=3 and x1>=-3) and (x2<=2 and x2>=-2):
            fitness.append(fitnes(x1,x2))
        else:
            fitness.append(0)
    return fitness

# PARENT SELECTION WITH ROULETTE WHEEL

In [15]:
#FUNGSI INI AKAN MENGEMBALIKKAN INDEX YANG NANTINYA 
def roulet(a):
    totalfitnes = 0
    for i in range(nInd):
        totalfitnes+=a[i]
    batas = random.uniform(0,1)
    idx = 0
    while(batas>0):
        batas -= a[idx]/totalfitnes
        idx+=1
    idx-=1
    return idx

# CROSS OVER SINGLE POINT

In [16]:
#FUNGSI INI AKAN MENGEMBALIKAN DUA ARRAY YANG DI CROSSOVER DARI PARENT1 DAN PARENT2
def crossover(parent1,parent2,nGen):
    probabilitas = 0.65 #65%
    rann = random.uniform(0,1)
    if rann <= probabilitas:
        batas = random.randint(1,(nGen-1))
        offspring1=[]
        offspring2=[]
        idx=0
        for i in range(batas):
            offspring1.append(parent1[i])
            offspring2.append(parent2[i])
            idx+=1
        for i in range(len(parent1)-batas):
            offspring1.append(parent2[idx])
            offspring2.append(parent1[idx])
            idx+=1
        return offspring1,offspring2
    else:
        return parent1,parent2

# MUTATION

In [17]:
#FUNGSI INI AKAN MENGEMBALIKAN DUA ARRAY YANG DI MUTASI DARI HASIL CROSSOVER
def mutasi(offspring1,offspring2):
    probabilitas = 0.01 #(1%)
    rann = random.uniform(0,1)
    mutasi1=[]
    mutasi2=[]
    if rann <= probabilitas:
        for i in range(len(offspring1)):
            if offspring1[i] == 0:
                mutasi1.append(1)
            elif offspring1[i] == 1:
                mutasi1.append(0)
        for i in range(len(offspring2)):
            if offspring2[i] == 0:
                mutasi2.append(1)
            elif offspring2[i] == 1:
                mutasi2.append(0)
        return mutasi1 , mutasi2
    else:
        return offspring1, offspring2

# SURVIVOR STEADY BY FITNESS

In [18]:
#FUNGSI INI AKAN MENGEMBALIKAN SEBUAH ARRAY YANG BERISI POPULASI BARU PADA GENERASI TERTENTU
def steadyfit(nInd,nGen,pop,offspring1,offspring2):
    popendc = PopEncoding(nInd,pop)
    popfitnes = hitungfitnes(popendc)
    idxmin=0
    idxmin2=0
    for i in range(len(popfitnes)):
        if popfitnes[idxmin]>popfitnes[i]:
            idxmin  = i
        
    for i in range(len(popfitnes)):
        if((i==idxmin)and(idxmin==0)):
            idxmin2 = idxmin2+1
        elif(i!=idxmin):
            if popfitnes[idxmin2]>popfitnes[i]:
                idxmin2 = i
    
    pop[idxmin] = copy.deepcopy(offspring1)
    pop[idxmin2] = copy.deepcopy(offspring2)
    return pop


# PROGRAM UTAMA


In [19]:
nInd = 10 #JUMLAH INDIVIDU
nGen = 8 #JUMLAH KROMOSOM SETIAP INDIVIDU
populasi1 = populasi(nInd,nGen) #POPULASI AWAL
jumGenerasi = 100 #JUMLAH GENERASI YANG AKAN DIBENTUK, KETIKA SUDAH KONVERGEN AKAN BERHENTI

pop = copy.deepcopy(populasi1) #POPULASI UNTUK MENCETAK GENERASI SELANJUTNYA
for i in range(jumGenerasi):
    #MENCARI FITNES DARI POPULASI TERBARU
    popendc = PopEncoding(nInd,pop)
    popfitnes = hitungfitnes(popendc)
    
    #CEK KONVERGEN, KETIKA KONVERGEN PERULANGAN AKAN BERHENTI
    cekkonvergen = copy.deepcopy(popfitnes[0])
    itungkonvergen = 0
    for j in range(len(popfitnes)-1):
        if cekkonvergen == popfitnes[j+1]:
            itungkonvergen +=1
    if itungkonvergen == (nInd-1):
        break
    
    #MENCARI PARENT
    parent1 = copy.deepcopy(pop[roulet(popfitnes)])
    parent2 = copy.deepcopy(pop[roulet(popfitnes)])
    
    #CROSSOVER
    popcross= crossover(parent1,parent2,nGen)
    
    #MUTATION
    popmutation = mutasi(popcross[0],popcross[1])
    
    #HASIL DARI CROOSOVER DAN MUTATION DARI PARENT1 DAN PARENT2
    offspring1 = copy.deepcopy(popmutation[0])
    offspring2 = copy.deepcopy(popmutation[1])
    
    
    #MENCARI GENERASI BARU DENGAN STEADY BY FITNES
    popgen = copy.deepcopy(steadyfit(nInd,nGen,pop,offspring1,offspring2))
    
    #POPULASI GENERASI TERBARU
    pop = copy.deepcopy(popgen)
    newencode = PopEncoding(nInd,pop)
    newfitnes = hitungfitnes(newencode)
    print("GENERASI KE-",i+1)
    print("POPULASI : ",pop)
    print("FITNES MASING-MASING INDIVIDU  : ",newfitnes)
    print()

print("THE BEST GENERATION")
print()
print("GENERASI TERBAIK/TERAKHIR/SUDAH KONVERGEN : ",pop)
print()
print("FITNES MASING-MASING INDIVIDU : ",newfitnes)
print()

#MENCARI FITNES TERBESAR DARI GENERASI TERAKHIR
idxmax = 0
for i in range(len(newfitnes)):
    if newfitnes[i]>newfitnes[idxmax]:
        idxmax=i

#ENCODE DARI INDIVIDU TERBAIK
bestInd = Encoding(pop[idxmax])

print("INDIVIDU TERBAIK : ",pop[idxmax])
print()
print("FITNES INDIVIDU TERBAIK/TERAKHIR/SUDAH KONVERGEN : ",newfitnes[idxmax])
print()
print("Nilai DARI X1 DAN X2 : ",bestInd[0],"DAN",bestInd[1])
print()
print("NILAI DARI FUNGSI : ",fungsi(bestInd[0],bestInd[1]))

GENERASI KE- 1
POPULASI :  [[1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 1, 0, 0], [1, 0, 0, 1, 1, 0, 1, 0], [1, 1, 0, 1, 0, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 0]]
FITNES MASING-MASING INDIVIDU  :  [0.2921754463199975, 0.24172775395595514, 0.9432647450213325, 0.3433658329800763, 0.14351869570209339, 0.6266694163785947, 0.11632829169772385, 0.2501380762180724, 0.6266694163785947, 0.8980844059196046]

GENERASI KE- 2
POPULASI :  [[1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 0]]
FITNES MASING-MASING INDIVIDU  :  [0.2921754463199975, 0.24172775395595514, 0.9432647450213325, 0.3433658329800763, 0.8980844059196046, 0.6266694163785947, 0.9432647450213325, 0.2501380

In [ ]:
def hitung_fitness(pop):
    fitness = []
    for i in range(len(pop)):
        if len(pop[i])==15:
            jumfit = 0
            for j in range(len(binary_data)):
                a = cp.deepcopy(pop[i])
                b = cp.deepcopy(binary_data[j])
                
                cat1krom = []
                cat2krom = []
                cat3krom = []
                cat4krom = []
                ############
                cat1data = []
                cat2data = []
                cat3data = []
                cat4data = []
                #############
                classkrom = a[-1]
                classdata = b[-1]
                #############
                hasiland = []
                hasilor = []
                
                for k in range(3):
                    cat1krom.append(a[k])
                    cat1data.append(b[k])
                hasiland.append(np.logical_and(cat1krom,cat1data))
                for k in range(3,7):
                    cat2krom.append(a[k])
                    cat2data.append(b[k])
                hasiland.append(np.logical_and(cat2krom,cat2data))
                for k in range(7,11):
                    cat3krom.append(a[k])
                    cat3data.append(b[k])
                hasiland.append(np.logical_and(cat3krom,cat3data))
                for k in range(11,14):
                    cat4krom.append(a[k])
                    cat4data.append(b[k])
                hasiland.append(np.logical_and(cat4krom,cat4data))
                
                for k in range(len(hasiland)):
                    for l in range(len(hasiland[k])):
                        if hasiland[k][l] == True:
                            hasilor.append(True)
                            break
                        if l == len(hasiland[k])-1 and hasiland[k][l]== False:
                            hasilor.append(False)
                
                hasiland2 = False
                
                for k in range(len(hasilor)):
                    if hasilor[k] == False:
                        hasiland2 = False
                        break
                    if k == len(hasilor)-1 and hasilor[k] == True:
                        hasiland2 = True
                
                if hasiland2 == True:
                    if classkrom == classdata:
                        jumfit += 1
                    else:
                        jumfit += 0
                else:
                    if classkrom != classdata:
                        jumfit += 1
                    else:
                        jumfit += 0
            fitness.append((jumfit/80)*100)
        else:
            panjangk = int(len(pop[i])/15)
            jumfit = 0
            for j in range(len(binary_data)):
                iterasi = 0
                for k in range(panjangk):
                    a = []
                    for l in range(iterasi,len(pop[i])):
                        a.append(pop[i][l])
                        iterasi+=1
                        if (l+1)%15==0:
                            break
                    b = cp.deepcopy(binary_data[j])
                    cat1krom = []
                    cat2krom = []
                    cat3krom = []
                    cat4krom = []
                    ############
                    cat1data = []
                    cat2data = []
                    cat3data = []
                    cat4data = []
                    #############
                    classkrom = a[-1]
                    classdata = b[-1]
                    #############
                    hasiland = []
                    hasilor = []
                    
                    for l in range(3):
                        cat1krom.append(a[l])
                        cat1data.append(b[l])
                    hasiland.append(np.logical_and(cat1krom,cat1data))
                    for l in range(3,7):
                        cat2krom.append(a[l])
                        cat2data.append(b[l])
                    hasiland.append(np.logical_and(cat2krom,cat2data))
                    for l in range(7,11):
                        cat3krom.append(a[l])
                        cat3data.append(b[l])
                    hasiland.append(np.logical_and(cat3krom,cat3data))
                    for l in range(11,14):
                        cat4krom.append(a[l])
                        cat4data.append(b[l])
                    hasiland.append(np.logical_and(cat4krom,cat4data))
                    
                    for l in range(len(hasiland)):
                        for m in range(len(hasiland[l])):
                            if hasiland[l][m] == True:
                                hasilor.append(True)
                                break
                            if m == len(hasiland[l])-1 and hasiland[l][m]== False:
                                hasilor.append(False)
                    
                    hasiland2 = False
                    
                    for l in range(len(hasilor)):
                        if hasilor[l] == False:
                            hasiland2 = False
                            break
                        if l == len(hasilor)-1 and hasilor[l] == True:
                            hasiland2 = True
                    
                    if hasiland2 == True:
                        if classkrom == classdata:
                            jumfit += 1
                            break
                        else:
                            jumfit += 0
                    else:
                        if classkrom != classdata:
                            jumfit += 1
                            break
                        else:
                            jumfit += 0
            fitness.append((jumfit/80)*100)        

    return fitness 


In [8]:
def hitung_fitness_lanjut(pop):
    fitness = []
    for i in range(len(pop)):
        if len(pop[i])==15:
            jumfit = 0
            for j in range(len(binary_data)):
                a = cp.deepcopy(pop[i])
                b = cp.deepcopy(binary_data[j])
                
                cat1krom = []
                cat2krom = []
                cat3krom = []
                cat4krom = []
                ############
                cat1data = []
                cat2data = []
                cat3data = []
                cat4data = []
                #############
                classkrom = []
                classdata = []
                classand = []
                classkrom = a[-1]
                classdata = b[-1]
                classand = np.logical_and(classkrom,classdata)
                #############
                hasiland = []
                hasilor = []
                
                for k in range(3):
                    cat1krom.append(a[k])
                    cat1data.append(b[k])
                hasiland.append(np.logical_and(cat1krom,cat1data))
                for k in range(3,7):
                    cat2krom.append(a[k])
                    cat2data.append(b[k])
                hasiland.append(np.logical_and(cat2krom,cat2data))
                for k in range(7,11):
                    cat3krom.append(a[k])
                    cat3data.append(b[k])
                hasiland.append(np.logical_and(cat3krom,cat3data))
                for k in range(11,14):
                    cat4krom.append(a[k])
                    cat4data.append(b[k])
                hasiland.append(np.logical_and(cat4krom,cat4data))
                
                for k in range(len(hasiland)):
                    for l in range(len(hasiland[k])):
                        if hasiland[k][l] == True:
                            hasilor.append(True)
                            break
                        if l == len(hasiland[k])-1 and hasiland[k][l]== False:
                            hasilor.append(False)
                hasilor.append(classand)
                
                hasiland2 = False
                
                for k in range(len(hasilor)):
                    if hasilor[k] == False:
                        hasiland2 = False
                        break
                    if k == len(hasilor)-1 and hasilor[k] == True:
                        hasiland2 = True
                
                
                if hasiland2 == True:
                    jumfit += 1
                else:
                    jumfit += 0
            fitness.append((jumfit/80)*100)
        else:
            panjangk = int(len(pop[i])/15)
            jumfit = 0
            for j in range(len(binary_data)):
                iterasi = 0
                for k in range(panjangk):
                    a = []
                    for l in range(iterasi,len(pop[i])):
                        a.append(pop[i][l])
                        iterasi+=1
                        if (l+1)%15==0:
                            break
                    b = cp.deepcopy(binary_data[j])
                    cat1krom = []
                    cat2krom = []
                    cat3krom = []
                    cat4krom = []
                    ############
                    cat1data = []
                    cat2data = []
                    cat3data = []
                    cat4data = []
                    #############
                    classkrom = []
                    classdata = []
                    classand = []
                    classkrom = a[-1]
                    classdata = b[-1]
                    classand = np.logical_and(classkrom,classdata)
                    #############
                    hasiland = []
                    hasilor = []
                    
                    for l in range(3):
                        cat1krom.append(a[l])
                        cat1data.append(b[l])
                    hasiland.append(np.logical_and(cat1krom,cat1data))
                    for l in range(3,7):
                        cat2krom.append(a[l])
                        cat2data.append(b[l])
                    hasiland.append(np.logical_and(cat2krom,cat2data))
                    for l in range(7,11):
                        cat3krom.append(a[l])
                        cat3data.append(b[l])
                    hasiland.append(np.logical_and(cat3krom,cat3data))
                    for l in range(11,14):
                        cat4krom.append(a[l])
                        cat4data.append(b[l])
                    hasiland.append(np.logical_and(cat4krom,cat4data))
                    
                    for l in range(len(hasiland)):
                        for m in range(len(hasiland[l])):
                            if hasiland[l][m] == True:
                                hasilor.append(True)
                                break
                            if m == len(hasiland[l])-1 and hasiland[l][m]== False:
                                hasilor.append(False)
                    hasilor.append(classand)
                    
                    hasiland2 = False
                    for l in range(len(hasilor)):
                        if hasilor[l] == False:
                            hasiland2 = False
                            break
                        if l == len(hasilor)-1 and hasilor[l] == True:
                            hasiland2 = True
                    
                    if hasiland2 == True:
                        jumfit += 1
                        break  
                    else:
                        jumfit += 0
            fitness.append((jumfit/80)*100)        
            ##########################################

    return fitness 
